In [ ]:
#r "nuget:Microsoft.ML"

Installed Packages Microsoft.ML, 1.6.0

In [ ]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
public class Features
{
    [LoadColumn(0)]
    public float FixedAcidity { get; set; }
    [LoadColumn(1)]
    public float VolatileAcidity { get; set; }
    [LoadColumn(2)]
    public float CitricAcid { get; set; }
    [LoadColumn(3)]
    public float ResidualSugar { get; set; }
    [LoadColumn(4)]
    public float Chlorides { get; set; }
    [LoadColumn(5)]
    public float FreeSulfurDioxide { get; set; }
    [LoadColumn(6)]
    public float TotalSulfurDioxide { get; set; }
    [LoadColumn(7)]
    public float Density { get; set; }
    [LoadColumn(8)]
    public float Ph { get; set; }
    [LoadColumn(9)]
    public float Sulphates { get; set; }
    [LoadColumn(10)]
    public float Alcohol { get; set; }
    [LoadColumn(11)]
    public float Quality { get; set; }
}

public class Label
{
    [ColumnName("PredictedLabel")]
    public float Quality { get; set; }
}

In [ ]:
var dataPath = "../datasets//winequality-red.csv";

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
var data = mlContext.Data.LoadFromTextFile<Features>(dataPath, hasHeader: true, separatorChar: ';');

In [ ]:
var partitions = mlContext.Data.TrainTestSplit(data, testFraction: 0.3);

var trainingDataView = partitions.TrainSet;
var testDataView = partitions.TestSet;

In [ ]:
var pipeline = mlContext.Transforms.Conversion.MapValueToKey(inputColumnName: "Quality", outputColumnName: "Label")
              .Append(mlContext.Transforms.Concatenate("Features", "FixedAcidity", "VolatileAcidity","CitricAcid", "ResidualSugar", 
                                                       "Chlorides", "FreeSulfurDioxide", "TotalSulfurDioxide","Density", "Ph", "Sulphates", "Alcohol"))
              .AppendCacheCheckpoint(mlContext);

In [ ]:
var trainer = mlContext.MulticlassClassification.Trainers.NaiveBayes("Label","Features");
var trainingPipeline = pipeline.Append(trainer).Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

In [ ]:
var trainedModel = trainingPipeline.Fit(partitions.TrainSet);
var predictions = trainedModel.Transform(partitions.TestSet);

var metrics = mlContext.MulticlassClassification.Evaluate(predictions);

In [ ]:
Console.WriteLine($"************************************************************");
Console.WriteLine($"*       MacroAccuracy: {metrics.MacroAccuracy:P2}");
Console.WriteLine($"*       MicroAccuracy: {metrics.MicroAccuracy:P2}");
Console.WriteLine($"************************************************************");

************************************************************
*       MacroAccuracy: 16,67%
*       MicroAccuracy: 40,53%
************************************************************
